### Constants Declaration

In [1]:
DATA_PATH = "../data/raw/survey_results_public.csv"
EXPORT_PATH = "../data/processed/first_preprocessed_df.pkl"

REPLACE_DICT = {
    'YearsCodePro': {'Less than 1 year': 0, 'More than 50 years': 51}, 
    'YearsCode':    {'Less than 1 year': 0, 'More than 50 years': 51}}

### Import the main Libraries

In [2]:
import pandas as pd 
import numpy as np
import logging
import pickle
pd.options.display.max_rows = 10000

### Functions

In [3]:
def split_answers(data_series, delimiter=";"):
    """ 
    Split multiple answers in a single string 
    to a list of single strings each represnting a single answers 

    Parameters:
    * data_series (pd.Series): String series with answers 
    * delimiter (string): Another decimal integer 
                          Defaults to ";"

    Returns: (pd.Series): If column contains 
    """
    # Sub functions 
    def is_splittable(pd_series, delimiter):
        """ Check if results multiple should be splitted - Returns boolean """    
        return pd_series.str.contains(delimiter)
    
    def split_answer(pd_series, delimiter):
        """Function to split single answer"""  
        return pd_series.str.split(delimiter)
    
    # Check if multiple answers exist - if none: return original 
    splittable_values = is_splittable(data_series, delimiter)
    if not splittable_values.any():
        return data_series
    
    # Else, split each value to a list 
    modified_series = split_answer(data_series, delimiter)    
    
    # Replace NAs with empty lists 
    mask_null = modified_series.isnull()
    modified_series.loc[mask_null] = modified_series.loc[mask_null].apply(lambda x: [])
    
    return modified_series

In [4]:
df = pd.read_csv(DATA_PATH)
df.shape

(83439, 48)

In [5]:
df.sample(1).iloc[0]

ResponseId                                                                   2013
MainBranch                                         I am a developer by profession
Employment                                                     Employed full-time
Country                                                  United States of America
US_State                                                               California
UK_Country                                                                    NaN
EdLevel                                                 Primary/elementary school
Age1stCode                                                           5 - 10 years
LearnCode                       Other online resources (ex: videos, blogs, etc...
YearsCode                                                                      13
YearsCodePro                                                                   13
DevType                         Developer, front-end;Developer, full-stack;Dev...
OrgSize         

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83439 entries, 0 to 83438
Data columns (total 48 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   ResponseId                    83439 non-null  int64  
 1   MainBranch                    83439 non-null  object 
 2   Employment                    83323 non-null  object 
 3   Country                       83439 non-null  object 
 4   US_State                      14920 non-null  object 
 5   UK_Country                    4418 non-null   object 
 6   EdLevel                       83126 non-null  object 
 7   Age1stCode                    83243 non-null  object 
 8   LearnCode                     82963 non-null  object 
 9   YearsCode                     81641 non-null  object 
 10  YearsCodePro                  61216 non-null  object 
 11  DevType                       66484 non-null  object 
 12  OrgSize                       60726 non-null  object 
 13  C

In [7]:
df.describe()

,ResponseId,CompTotal,ConvertedCompYearly
count,83439.000000,4.718300e+04,4.684400e+04
mean,41720.000000,2.119407e+69,1.184262e+05
std,24086.908893,4.603702e+71,5.272944e+05
min,1.000000,0.000000e+00,1.000000e+00
25%,20860.500000,1.600000e+04,2.702500e+04
50%,41720.000000,6.700000e+04,5.621100e+04
75%,62579.500000,1.400000e+05,1.000000e+05
max,83439.000000,1.000000e+74,4.524131e+07


In [8]:
questionable_cols = ['YearsCodePro', 'YearsCode']

for col in questionable_cols:
    print(col)
    print(df[col].unique().tolist())
    print('--------------------------')
    print()

YearsCodePro
[nan, '10', '4', '5', '6', '2', '30', '9', '18', '12', '21', '1', '16', 'Less than 1 year', '15', '3', '35', '7', '8', '17', '14', '26', '25', '20', '50', '34', '11', '24', '22', '13', '31', '23', '39', '41', '27', '28', '19', '33', 'More than 50 years', '37', '29', '32', '43', '40', '38', '45', '42', '46', '36', '44', '47', '48', '49']
--------------------------

YearsCode
[nan, '7', '17', '3', '4', '6', '16', '12', '15', '10', '40', '9', '26', '14', '39', '20', '8', '19', '5', 'Less than 1 year', '22', '2', '1', '34', '21', '13', '25', '24', '30', '31', '18', '38', 'More than 50 years', '27', '41', '42', '35', '23', '28', '11', '37', '44', '43', '36', '33', '45', '29', '50', '46', '32', '47', '49', '48']
--------------------------



### Preprocess Data

In [9]:
df_copy = df.copy()

**1. Replace Values and parse**

In [10]:
for col, replacement in REPLACE_DICT.items():
    df_copy[col] = df_copy[col].replace(replacement).astype(np.float32)

**2. Split multiple answers**

In [11]:
object_cols = df_copy.select_dtypes(include='object').columns.tolist()
for col in object_cols:
    df_copy[col] = split_answers(df_copy[col])

In [12]:
i = df.sample(1).index[0]
print(df['LanguageHaveWorkedWith'].iloc[i])
print(df_copy['LanguageHaveWorkedWith'].iloc[i])

C;C++;HTML/CSS;Java;JavaScript;SQL
['C', 'C++', 'HTML/CSS', 'Java', 'JavaScript', 'SQL']


In [13]:
i = df.sample(1).index[0]
print(df['DevType'].iloc[i])
print(df_copy['DevType'].iloc[i])

Developer, full-stack;Student
['Developer, full-stack', 'Student']


In [14]:
i = df.sample(1).index[0]
print(df['YearsCodePro'].iloc[i])
# nulls = df_copy['YearsCodePro'].isnull()
# df_copy['YearsCodePro'].loc[nulls] = df_copy['YearsCodePro'].loc[nulls].apply(lambda x: [])
print(df_copy['YearsCodePro'].iloc[i])

4
4.0
